In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [16]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 6
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/230216_tdcb/'
sp_plasmids_file = folder + '230216_plasmids_sp.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'genex-pl-96.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'genex_mt.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'230207_plasmids_sp_updated.xlsx')
protocol_genex_df = protocol_genex_df[0]


Volumes used from each well for this protocol:
{'1_A1': 12.325, '1_P1': 0.25, '1_P2': 0.375, '1_P3': 0.1, '1_P5': 0.1, '1_P8': 2.175, '1_P9': 2.55, '1_O16': 0.125}


In [17]:
vol_table_df

,Label,water,biotin,Pyc,Fdh,P70_GFP,Mdh,Mdh_10x,ME2,TdcB,...,gcvH,lpd,lplA,fdhCB,glyA,TdcB_2,FAD,fch_10x,ftl_10x,PLP_10x
0,tdcb_10,0.6,0,0,0,NaN,0,NaN,NaN,0.775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tdcb_5,0.975,0,0,0,NaN,0,NaN,NaN,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tdcb_2.5,1.175,0,0,0,NaN,0,NaN,NaN,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tdcb_.5,1.325,0,0,0,NaN,0,NaN,NaN,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tdcb_0,1.375,0,0,0,NaN,0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PLP_10,0.15,0,0,0,NaN,0,NaN,NaN,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PLP_5,0.75,0,0,0,NaN,0,NaN,NaN,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,PLP_1,1.225,0,0,0,NaN,0,NaN,NaN,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PLP_.1,1.225,0,0,0,NaN,0,NaN,NaN,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125
9,PLP_0,1.35,0,0,0,NaN,0,NaN,NaN,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'P23'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

tdcb_10
PLP_10
tdcb_5
PLP_5
tdcb_2.5
PLP_1
tdcb_.5
PLP_.1
tdcb_0
PLP_0
pyc
pyc_neg
TXTL needed: 54.0


In [6]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [7]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [21]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
#sp_genex = sp_from_layout(layout_genex, 60)
layout_genex_384_file = folder + 'genex-pl-384.csv'
layout_genex_384 = pd.read_csv(layout_genex_384_file, index_col = 0, dtype = str)
sp_genex = sp_from_layout(layout_genex_384, 60)


# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'buffers-mt.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 3.0, '1_B1': 3.0, '1_C1': 1.5, '1_D1': 3.0, '1_E1': 15.0, '1_F1': 2.025, '1_G1': 2.4, '1_H1': 6.0, '1_I1': 41.25, '1_I2': 6.0, '1_J1': 3.0, '1_K1': 6.0, '1_N1': 39.15, '1_O1': 28.5, '1_P1': 41.0, '1_P2': 22.75, '2_A2': 1036.425, '3_A13': 7.5, '3_B13': 7.5, '3_C13': 40.0, '3_C14': 35.0, '3_D13': 7.5, '3_E13': 7.5, '3_F13': 7.5, '3_G13': 7.5, '3_H13': 7.5, '3_I13': 7.5, '3_J13': 7.5, '3_K13': 7.5, '3_L13': 7.5}


In [22]:
protocol_biosyn_dfs[0].to_csv(folder+'biosyn_protocol_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'biosyn_protocol_6RES.csv',index = False)

In [20]:
sps[2]

,Concentration,Label,Volume,Well
Item,,,,
tdcb_10,100.0,tdcb_10,"55,55","A13,A14"
tdcb_5,100.0,tdcb_5,"55,55","B13,B14"
tdcb_2.5,100.0,tdcb_2.5,"55,55","C13,C14"
tdcb_.5,100.0,tdcb_.5,"55,55","D13,D14"
tdcb_0,100.0,tdcb_0,"55,55","E13,E14"
PLP_10,100.0,PLP_10,"55,55","F13,F14"
PLP_5,100.0,PLP_5,"55,55","G13,G14"
PLP_1,100.0,PLP_1,"55,55","H13,H14"
PLP_.1,100.0,PLP_.1,"55,55","I13,I14"
